In [2]:
###################################################################
# This is an official PyTorch implementation for Target-specific
# Generation of Molecules (TagMol)
# Author: Junde Li, The Pennsylvania State University
# Date: Aug 1, 2022
###################################################################

import os, time
import re
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from rdkit import Chem

In [7]:
# --------------------------
# Hyperparameters
# --------------------------

lr             = 1e-4
beta1          = 0.0
beta2          = 0.9
batch_size     = 16
max_epoch      = 400
num_workers    = 2
ligand_size    = 32
x_size         = 128
z_size         = 128
save_step      = 100

name           = "model/tagmol"
log_dir        = f"{name}"
models_dir     = f"{name}/saved_models"
device         = torch.device("cuda:0")

os.makedirs(log_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)